<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model_regression_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install python-chess

In [2]:
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data_with_buckets.csv"
csv_path = f"{folder}/{csv_file}"


now = datetime.now()
dt_string = now.strftime("%m_%d_%Y_%H_%M")

model_file = "regression_model.pth"
datetime = dt_string
model_path = f"{folder}/{datetime}_{model_file}"

Mounted at /content/gdrive


In [3]:
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from imblearn.under_sampling import RandomUnderSampler

###Parameters

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    "batch_size": 1,
    "epochs": 5,
    "learning_rate": 0.00003,
    "warmup_steps": 0.01,
    "epsilon": 1e-8,
    "accum_iter": 8,
    "num_classes": 6,
    "dropout": 0.5
}

class Params:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

params = Params(**params)

### Regression Model

In [5]:
class BertRegressor(nn.Module):
  def __init__(self, dropout=params.dropout):
    super(BertRegressor, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, 1)

  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)

    return linear_output

### Dataset Class

In [6]:
class ChessDataset(torch.utils.data.Dataset):
  def __init__(self, moves, labels):
      tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      self.labels = torch.Tensor(labels)
      self.moves = [tokenizer(move, 
                              padding='max_length', max_length = 256, truncation=True,
                              return_tensors="pt") for move in moves]

  def __len__(self):
      return len(self.labels)

  def __getitem__(self, idx):
      return self.moves[idx], self.labels[idx]

Helper Functions

In [7]:
def rating_to_bucket(rating):
  if rating < 1300:
    return 0
  elif rating >= 1300 and rating < 1500:
    return 1
  elif rating >= 1500 and rating < 1700:
    return 2  
  elif rating >= 1700 and rating < 1900:
    return 3
  elif rating >= 1900 and rating < 2100:
    return 4
  elif rating >= 2100:
    return 5

In [8]:
def get_metrics(model_output, label):
  """
  Returns accuracy and loss of true ELO values
  """
  
  output = model_output.cpu().detach().numpy()
  label = label.cpu().detach().numpy()
  
  pred_rating = torch.Tensor(scaler.inverse_transform(output))
  real_rating = torch.Tensor(scaler.inverse_transform(label))
  loss = criterion(pred_rating, real_rating)

  pred_bucket = np.array(list(map(rating_to_bucket, pred_rating)))
  real_bucket = np.array(list(map(rating_to_bucket, real_rating)))

  acc = (pred_bucket == real_bucket).sum().item()
  return acc, loss

### Training Function

In [9]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer):
    num_batches = len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = params.warmup_steps, num_training_steps = num_batches * params.epochs)
    print(f"Number of batches: {num_batches}")
    for epoch_num in range(params.epochs):
        total_acc_train = 0
        total_loss_train = 0

        for batch_idx, (train_input, train_label) in enumerate(train_dataloader):
          train_label = train_label.to(device)
          mask = train_input['attention_mask'].to(device)
          input_id = train_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)
          
          batch_loss = criterion(output.squeeze(), train_label.squeeze())
          total_loss_train += batch_loss.item()
          
          acc, _ = get_metrics(output, train_label)
          total_acc_train += acc

          batch_loss.backward()

          # Gradient Accumulation
          if ((batch_idx + 1) % params.accum_iter == 0) or (batch_idx + 1 == num_batches):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

          if batch_idx % 50 == 0:
            print(f"Batch Number: {batch_idx}")
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

          for val_input, val_label in val_dataloader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()
            
            acc, _ = get_metrics(output, val_label)
            total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / (params.batch_size * len(train_dataloader)): .3f} \
            | Train Accuracy: {total_acc_train / (params.batch_size * len(train_dataloader)): .3f} \
            | Val Loss: {total_loss_val / (params.batch_size * len(val_dataloader)): .3f} \
            | Val Accuracy: {total_acc_val / (params.batch_size * len(val_dataloader)): .3f}')
        
        torch.save(model.state_dict(), model_path)

### Test Function

In [10]:
def test(model, test_dataloader, criterion, scaler):
  total_acc_test = 0
  total_loss_test = 0
  with torch.no_grad():
    for test_input, test_label in test_dataloader:
      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)
      
      acc, loss = get_metrics(output, test_label)
      total_acc_test += acc
      total_loss_test += loss
       
    print(f"Test Accuracy: {total_acc_test / len(test_dataloader): .3f}")
    print(f"Test Loss: {total_loss_test / len(test_dataloader): .3f}")

### Loading Data

In [11]:
%%capture
chess_data = pd.read_csv(csv_path)[0:30000]
z = chess_data["Bucket"]
chess_data = chess_data[["Moves", "Elo"]]

balancer = RandomUnderSampler()
chess_data, z = balancer.fit_resample(chess_data, z)
rating_counts = z.value_counts().sort_index()

X = chess_data["Moves"]
y = chess_data["Elo"]

scaler = StandardScaler()
y = scaler.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, shuffle=True) # Splits into train/test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle=True) # Splits train into train/val

train_data, val_data, test_data = ChessDataset(X_train, y_train), ChessDataset(X_val, y_val), ChessDataset(X_test, y_test)

In [12]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=params.batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=params.batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data)

### Initialize Model

In [13]:
%%capture
model = BertRegressor()
model = model.to(device)
criterion = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr = params.learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Run Training

In [14]:
train(model, train_dataloader, val_dataloader, criterion, optimizer)

Number of batches: 6336
Batch Number: 0
Batch Number: 50
Batch Number: 100
Batch Number: 150
Batch Number: 200
Batch Number: 250
Batch Number: 300
Batch Number: 350
Batch Number: 400
Batch Number: 450
Batch Number: 500
Batch Number: 550
Batch Number: 600
Batch Number: 650
Batch Number: 700
Batch Number: 750
Batch Number: 800
Batch Number: 850
Batch Number: 900
Batch Number: 950
Batch Number: 1000
Batch Number: 1050
Batch Number: 1100
Batch Number: 1150
Batch Number: 1200
Batch Number: 1250
Batch Number: 1300
Batch Number: 1350
Batch Number: 1400
Batch Number: 1450
Batch Number: 1500
Batch Number: 1550
Batch Number: 1600
Batch Number: 1650
Batch Number: 1700
Batch Number: 1750
Batch Number: 1800
Batch Number: 1850
Batch Number: 1900
Batch Number: 1950
Batch Number: 2000
Batch Number: 2050
Batch Number: 2100
Batch Number: 2150
Batch Number: 2200
Batch Number: 2250
Batch Number: 2300
Batch Number: 2350
Batch Number: 2400
Batch Number: 2450
Batch Number: 2500
Batch Number: 2550
Batch Numbe

### Run Testing

In [15]:
# test_model_name = "06_01_2023_20_27_regression_model.pth"
# test_model_path = f"{folder}/{test_model_name}"

test_model = BertRegressor()
test_model = test_model.to(device)
test_model.load_state_dict(torch.load(model_path))
test(test_model, test_dataloader, criterion, scaler)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test Accuracy:  0.178
Test Loss:  293.247
